In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession

sc = SparkContext("local", "pbl-1")
spark = SparkSession(sc)

In [35]:
data = sc.textFile("facebook_combined.txt")

parse_data = data.flatMap(lambda line: line.split('\n')) \
            .map(lambda l:l.split(' ')).map(lambda l: [(int(l[0]),int(l[1])),(int(l[1]),int(l[0]))]).flatMap(lambda t: [t[0],t[1]])

In [36]:
group = parse_data.groupByKey().mapValues(list)
new_group = group.flatMap(lambda l: [(l[0],(len(l[1]), sorted(l[1])))])

In [37]:
order = new_group.sortBy(lambda v: v[1])
R = order.mapValues(lambda v: v[1]).collect()

In [ ]:
import math
import time

r_set = [0.6, 0.7, 0.8, 0.9]
set_idx = {id:i for i,(id,x) in enumerate(R)}


for r in r_set:
    print("# when r = ",r)
    start = time.time()
    
    S = []
    I = [[] for _ in range(len(R))]

    for (x_id, x) in R: # 순서대로 inverted list를 작성해야 하므로 동기 계산 필요
        A = [0 for _ in range(len(R))]
        
        max_probe_prefix = len(x) - math.ceil(r*len(x)) + 1
        max_index_prefix = len(x) - math.ceil(2*len(x)*r/(r+1)) + 1
        
        prefix_x = sc.parallelize([i for i in range(max_probe_prefix+1)]) \
                    .map(lambda t: len(x)-(math.ceil((2*len(x)-t+1)*r/(r+1)))+1).collect()
        prefix_y = sc.parallelize([i for i in range(max_probe_prefix+1)]) \
                    .map(lambda t: len(x)-t+1-(math.ceil((2*len(x)-t+1)*r/(r+1)))+1).collect()
        

        for i in range(1,max_probe_prefix+1):
            w = x[i-1]
            w_idx = set_idx[w]

            inverted_list = sc.parallelize(I[w_idx]).filter(lambda t: len(t[0][1]) >= r*len(x) and t[1] <= prefix_y[len(x)-len(t[0][1])+1])
            I[w_idx] = inverted_list.collect()
            
            candidate = inverted_list.filter(lambda t: i <= prefix_x[len(x)-len(t[0][1])+1]).collect()
            for item in candidate:
                item_id = item[0][0]
                A[set_idx[item_id]] = A[set_idx[item_id]]+1

        for i in range(1,max_probe_prefix+1):
            w = x[i-1]
            w_idx = set_idx[w]

            for ((y_id, y),j) in I[w_idx]:
                if len(y) < r*len(x):
                    I[w_idx].remove(((y_id,y),j))
                elif j > prefix_y[len(x)-len(y)+1]:
                    I[w_idx].remove(((y_id,y),j))
                elif i > prefix_x[len(x)-len(y)+1]: continue
                else:
                    A[set_idx[y_id]] = A[set_idx[y_id]] + 1 # 후보 확정

        for i in range(1,max_index_prefix+1): # x의 원소에 대한 inverted list 추가
            w = x[i-1]
            w_idx = set_idx[w]
            I[w_idx].append(((x_id,x),i))
                
        for i in range(len(R)): # 후보 검증 후 S에 추가
            (y_id, y) = R[i]

            if A[i] == 0: continue # prefix 안에 공통 토큰이 하나도 없다면 제외

            a = math.ceil((len(x)+len(y))*r/(r+1))

            (p,q,cnt) = (0,0,0)
            
            while(p<len(x) and q<len(y)):# 공통 토큰 개수 카운트
                if x[p] == y[q]:
                    cnt = cnt+1
                    p = p+1
                    q = q+1
                elif cnt+min(len(x)-p-1,len(y)-q-1) < a: break # positional filtering
                elif x[p] < y[q]: p = p+1
                else: q = q+1
            if cnt >= a and y_id not in x: # x와 y가 이미 친구인 경우는 제외
                S.append((x_id, y_id))

    end = time.time()

    print(S) 
    print("duration : ", end-start)

# when r =  0.6
[(12, 11), (15, 11), (15, 12), (18, 11), (18, 12), (18, 15), (37, 11), (37, 12), (37, 15), (37, 18), (43, 11), (43, 12), (43, 15), (43, 18), (43, 37), (74, 11), (74, 12), (74, 15), (74, 18), (74, 37), (74, 43), (114, 11), (114, 12), (114, 15), (114, 18), (114, 37), (114, 43), (114, 74), (209, 11), (209, 12), (209, 15), (209, 18), (209, 37), (209, 43), (209, 74), (209, 114), (210, 11), (210, 12), (210, 15), (210, 18), (210, 37), (210, 43), (210, 74), (210, 114), (210, 209), (215, 11), (215, 12), (215, 15), (215, 18), (215, 37), (215, 43), (215, 74), (215, 114), (215, 209), (215, 210), (287, 11), (287, 12), (287, 15), (287, 18), (287, 37), (287, 43), (287, 74), (287, 114), (287, 209), (287, 210), (287, 215), (292, 11), (292, 12), (292, 15), (292, 18), (292, 37), (292, 43), (292, 74), (292, 114), (292, 209), (292, 210), (292, 215), (292, 287), (335, 11), (335, 12), (335, 15), (335, 18), (335, 37), (335, 43), (335, 74), (335, 114), (335, 209), (335, 210), (335, 215), (335, 